In [18]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, models, losses, optimizers
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [19]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_ds = train_gen.flow_from_directory(
    './data/train/',
    target_size=(200, 200), # Size of the image
    batch_size=32  # How many image at once we need
)

test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_ds = test_gen.flow_from_directory(
    './data/train/',
    target_size=(200, 200), # Size of the image
    batch_size=32  # How many image at once we need
)

Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [ ]:
def make_model(
                filters = 32,
                dense_inner_size = 64,
                lr = 0.002,
                momentum = 0.8,
                loss = tf.keras.losses.BinaryCrossentropy()
                ):
    inputs = keras.Input(shape=(200, 200, 3))
    
    # Convolutional layers
    cvl_layers = Conv2D(filters=filters, kernel_size=(3, 3), activation='relu')(inputs)
    feature_map = MaxPooling2D(pool_size=(2, 2))(cvl_layers)
    
    # Flatten layer
    vectors = Flatten()(feature_map)
    
    # Dense layers
    dense_layers = Dense(dense_inner_size, activation='relu')(vectors)
    
    # Output layer for binary classification
    outputs = Dense(1, activation='sigmoid')(dense_layers)
    
    # Model construction
    model = keras.Model(inputs, outputs)
    
    # Optimizer and compiling the model
    optimizer = SGD(learning_rate=lr, momentum=momentum)
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )
    
    return model

In [21]:
model = make_model()

In [22]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 99, 99, 32)       0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 313632)            0         
                                                                 
 dense_8 (Dense)             (None, 64)                20072512  
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
__________________________________________